In [1]:
import pandas as pd
from collections import Counter
import numpy as np
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler

from sklearn.preprocessing import MinMaxScaler

import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv,ResGatedGraphConv,GATv2Conv,SAGEConv,GCNConv
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.loader import NeighborSampler, NeighborLoader
import copy
import tqdm

import torch
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.data import Data
from torch_geometric.utils.convert import to_networkx, from_networkx
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [2]:
df = pd.read_pickle('cicids2017_not_contain_monday_max.pkl')

In [3]:
x = df.drop(columns=[' Label'])
y = df[' Label']

In [4]:
rus = RandomUnderSampler(random_state=0, sampling_strategy={
    'BENIGN': 20000,
     'DDoS': 10000,
     'PortScan': 10000,
     'Bot': 1966,
     'Infiltration': 36,
     'FTP-Patator': 7938,
     'SSH-Patator': 5897,
     'DoS slowloris': 5796,
     'DoS Slowhttptest': 5499,
     'DoS Hulk': 10000,
     'DoS GoldenEye': 10293,
     'Heartbleed': 11
})
LABELS_12 = {
    'BENIGN': 0,
     'DDoS': 1,
     'PortScan': 2,
     'Bot': 3,
     'Infiltration': 4,
     'FTP-Patator': 5,
     'SSH-Patator': 6,
     'DoS slowloris': 7,
     'DoS Slowhttptest': 8,
     'DoS Hulk': 9,
     'DoS GoldenEye': 10,
     'Heartbleed': 11
}

In [5]:
x, y = rus.fit_resample(x, y)

In [6]:
label = [LABELS_12[v] for v in y]

In [7]:
df = x
df[' Label'] = label

In [8]:
df = df.dropna()

In [10]:
label = df[' Label']

In [11]:
x = df.drop(columns=[' Label', 'Flow ID',' Source IP',' Source Port', ' Destination IP', ' Destination Port',' Timestamp'])

In [12]:
x = x.replace(['Infinity','infinity ','inf', '-inf','INF','NaN','nan'], 0)

In [13]:
x = (x - x.min())/(x.max()-x.min())

In [15]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
mms = MinMaxScaler()
x_new = mms.fit_transform(x)

D:\anaconda\envs\lightgbm_catboost\lib\site-packages\sklearn\preprocessing\_data.py:400: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
D:\anaconda\envs\lightgbm_catboost\lib\site-packages\sklearn\preprocessing\_data.py:401: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [19]:
df2 = pd.DataFrame(x_new)
df2 = df2.fillna(0)

In [20]:
features = np.array(df2)

In [21]:
contain_nan = (True in np.isnan(features))
contain_nan

False

In [22]:
src_ip = df[' Source IP']
dst_ip = df[' Destination IP']
src_port = df[' Source Port']
dst_port = df[' Destination Port']

In [24]:
def process(src_ip, src_port, dst_ip, dst_port):
    def build_edge_index(arr):
        for i in range(len(arr)):
            if arr[i] not in adj_list:
                adj_list[arr[i]] = set()
            for j in range(len(arr)):
                if i != j:
                    adj_list[arr[i]].add(arr[j])
    nodes_to_no_dict = {}
    socket_adj_map = {}
    socket_no_map = {}
    adj_list = {}
    src_ip = np.array(src_ip)
    dst_ip = np.array(dst_ip)
    src_port = np.array(src_port)
    dst_port = np.array(dst_port)
    src_nodes = list(zip(src_ip, src_port))
    dst_nodes = list(zip(dst_ip, dst_port))
    socket_dict = list(zip(src_nodes, dst_nodes))
    all_nodes = set(src_nodes).union(set(dst_nodes))
    print(len(all_nodes))
    for index,value in enumerate(all_nodes):
        nodes_to_no_dict[value] = index
    num_edge = len(src_nodes)
    for index, value in enumerate(socket_dict):
        # 获取发出者编号
        s, d = value[0], value[1]
        s_no = nodes_to_no_dict[s]
        d_no = nodes_to_no_dict[d]
        if s_no not in socket_adj_map:
            socket_adj_map[s_no] = []
        if s_no not in socket_no_map:
            socket_no_map[s_no] = []
        socket_adj_map[s_no].append(d_no)
        socket_no_map[s_no].append(index)
    for index, value in socket_no_map.items():
        build_edge_index(value)
    for key, value in socket_adj_map.items():
        for i in range(len(value)):
            adj_no = value[i] 
            no = socket_no_map[key][i]
            if adj_no in socket_adj_map and len(socket_adj_map[adj_no]) != 0:
                adj_values = socket_adj_map[adj_no]
                for j in range(len(adj_values)):
                    if adj_values[j] == key:
                        a = no
                        b = socket_no_map[adj_no][j]
                        adj_list[a].add(b)
                        adj_list[b].add(a)
    return adj_list


def get_torch_data(adj_list):
    src_index = []
    dst_index = []
    for src, dst_arr in adj_list.items():
        for dst in dst_arr:
            src_index.append(src)
            dst_index.append(dst)
    edge_index = [src_index, dst_index]
    return edge_index

In [26]:
adj_list = process(src_ip, src_port, dst_ip, dst_port)

3945


KeyboardInterrupt: 